In [2]:
#import packages
from functools import partial
import numpy as np
from numpy import random
from skimage import exposure, img_as_float
from skimage.transform import resize
from saliency.metrics import NSS
import matlab
import matlab.engine
import pandas as pd
import numpy as np
import os
import cv2
import saliency
import glob
import matplotlib
from matplotlib import pyplot as plt
from saliency.metrics import NSS, AUC, IG, CC, SAUC, SIM, KLdiv

In [7]:
#read in total data (fixations & saccades) with downsampled year 2000
df_total = pd.read_csv('Data/total_data.csv')

In [8]:
#check if any fixation x coordinates fall outside screen
df_total[df_total['end_x'] > 1920].DoB

3056     1990
3057     1990
3068     1990
3069     1990
3406     2001
6820     1998
7119     1968
8934     2003
8935     2003
10976    2002
Name: DoB, dtype: int64

In [9]:
#check if any fixation y coordinates fall outside screen

df_total[df_total['end_y'] > 1079]

,Unnamed: 0,Unnamed: 0_x,label,onset,duration,amp,avg_vel,med_vel,peak_vel,start_x,...,ind_id_fn,ID,Gender,DoB,ROI1,ROI2,ROI3,Version,Valid,Valid Freeviewing
112,26516,8,FIXA,4.087,0.567,346.010,54717.697,54778.302,91703.306,214.9,...,20210711131101810,202107111311018,MALE,1996,Pierce,Man,Octopus,2,True,True
316,27441,22,FIXA,6.740,2.269,332.782,12820.880,6226.109,50832.049,188.6,...,20210713114938710,202107131149387,MALE,2000,Turtle,Spaceship,FatherSonBoat,2,True,True
1848,36534,8,FIXA,2.402,0.350,284.190,71293.982,51205.987,150860.829,72.1,...,20210802110742853,202108021107428,MALE,2000,Pierce,WomanOnBoat,Clouds,2,True,True
1871,36557,12,FIXA,3.702,1.002,375.548,32280.749,28676.652,91910.904,136.9,...,20210802114425753,202108021144257,MALE,1972,BoatSunset,Turtle,Man,2,True,True
2326,39288,12,FIXA,6.789,1.102,875.914,110390.370,112918.598,197848.978,714.4,...,20210809123019242,202108091230192,MALE,2014,BoatSunset,WomanOnBoat,Man,2,True,True
2539,40478,4,FIXA,3.203,0.317,226.303,50935.263,45588.894,92304.457,38.2,...,20210812092910411,202108120929104,MALE,2005,Turtle,Elephant,Octopus,2,True,True
2707,41527,6,FIXA,3.736,1.936,83.934,4953.497,4649.358,13877.457,325.1,...,20210814154611123,202108141546111,FEMALE,2010,Elephant,WomanOnBoat,Turtle,2,True,True
3056,44075,1,SACC,0.233,0.250,1353.387,322302.379,309378.370,567772.768,887.3,...,20210821155006996,202108211550069,FEMALE,1990,WomanOnBoat,FatherSonBoat,Spaceship,2,True,True
3057,44076,2,FIXA,0.483,0.084,144.071,111887.897,112763.258,143450.695,2139.2,...,20210821155006996,202108211550069,FEMALE,1990,WomanOnBoat,FatherSonBoat,Spaceship,2,True,True
3068,44087,13,SACC,3.003,0.267,1217.860,271393.839,247686.982,486496.121,1616.1,...,20210821155006996,202108211550069,FEMALE,1990,WomanOnBoat,FatherSonBoat,Spaceship,2,True,True


In [10]:
#filter to only fixations
df_total = df_total[df_total.label == 'FIXA']

In [11]:
#get ids of those participants whose fixations fall outside screen
id_errors = df_total[(df_total.end_x > 1920) | (df_total.end_y > 1079)].ID.unique()

In [12]:
#check length
len(id_errors)

39

In [13]:
#create new dataset that excludes participants with fixations outside screen
df_corr = df_total[df_total.ID.isin(id_errors) == False]

In [14]:
#create list of unique ids that will be included in final dataset
ids = df_corr.ID.unique()

In [15]:
#create list of fixations for each participant

#create empty list
fixations = []

#loop over ids
for i in ids:
    #filter dataframe on current id
    df_temp = df_corr[df_corr.ID == i]
    #create temporary list
    temp_fix = []
    #loop over each row in temp dataframe
    for row in df_temp.itertuples():
        #select end x and y fixation coordinates and append to list
        my_list = [row.end_x,row.end_y]
        temp_fix.append(my_list)
    #append fixations and id to final list
    fixations.append([i,temp_fix])

In [16]:
#create dataframe of fixations and id
df_fix = pd.DataFrame(fixations, columns = ['id','fixations'])

In [17]:
#create fixation map for each participant, with a 1 at every fixation location, 0 everywhere else 
fix_map_col = []

#loop over every participant
for row in df_fix.itertuples():
    fixations = row.fixations
    #create an empty fixation map of all zeros
    fix_map = np.zeros((1079,1920))
    #add a 1 at every fixation location
    for x,y in fixations:
        x = round(x)
        y=round(y)
        fix_map[y-1][x-1] = 1
    fix_map_col.append(fix_map)

In [18]:
#add fixation map to dataframe
df_fix['fixmap'] = fix_map_col

In [19]:
df_fix.head()

,id,fixations,fixmap
0,202107101214363,"[[288.8, 199.7], [1292.7, 915.9], [354.9, 946....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,202107101432200,"[[891.4, 672.8], [1223.4, 744.5], [956.8, 167....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,202107101444503,"[[848.5, 851.7], [1400.1, 871.9], [1472.3, 295...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,202107101528378,"[[908.6, 304.9], [238.5, 226.8], [1354.1, 827....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,202107111045352,"[[921.4, 688.4], [117.9, 985.6], [211.4, 463.1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [20]:
f = []
for row in df_corr.itertuples():
    my_list = [row.end_x,row.end_y]
    f.append(my_list)

In [21]:
f_map = np.zeros((1079,1920))

for x,y in f:
    x = round(x)
    y = round(y)
    f_map[y-1][x-1] = 1

In [22]:
kernel = np.ones((11,9),np.float32)/25
dst = cv2.filter2D(f_map,-1,kernel)

In [24]:
#read in aim saliency map
aim_sal_g = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/aim_sal.png',0)

In [27]:
#calculate nss for each participant with aim
aim_nss = []
for i in range(len(df_fix)):
    x = NSS(aim_sal_g,df_fix.iloc[i][2])
    aim_nss.append(x)

df_fix['aim_nss'] = aim_nss

In [28]:
#calculate auc for each participant with aim

aim_auc = []
for i in range(len(df_fix)):
    x = AUC(aim_sal_g,df_fix.iloc[i][2])
    aim_auc.append(x)

df_fix['aim_auc'] = aim_auc

In [29]:
#calculate ig for each participant with aim

aim_ig = []
for i in range(len(df_fix)):
    x = IG(aim_sal_g,df_fix.iloc[i][2],dst)
    aim_ig.append(x)

df_fix['aim_ig'] = aim_ig

In [30]:
#calculate ig for each participant with aim

aim_cc = []
#kernel = np.ones((11,9),np.float32)/25

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    x = CC(aim_sal_g,blur)
    aim_cc.append(x)

df_fix['aim_cc'] = aim_cc

In [31]:
#calculate kl for each participant with aim

aim_kl = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    x = KLdiv(aim_sal_g,blur)
    aim_kl.append(x)

df_fix['aim_kl'] = aim_kl

In [32]:
#calculate sauc for each participant with aim

aim_sauc = []

for i in range(len(df_fix)):
    x = SAUC(aim_sal_g,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    aim_sauc.append(x)

df_fix['aim_sauc'] = aim_sauc

In [33]:
#calculate sim for each participant with aim

aim_sim = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    x = SIM(aim_sal_g,blur)
    aim_sim.append(x)

df_fix['aim_sim'] = aim_sim

In [34]:
#compare each participants fixations to rare saliency map and store output in dataframe
rare_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/rare_sal.png',0)
rare = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(rare_sal,df_fix.iloc[i][2])
    b = AUC(rare_sal,df_fix.iloc[i][2])
    c = IG(rare_sal,df_fix.iloc[i][2],dst)
    d = CC(rare_sal,blur)
    e = KLdiv(rare_sal,blur)
    f = SAUC(rare_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(rare_sal,blur)
    rare.append([a,b,c,d,e,f,g])

df_rare = pd.DataFrame(rare, columns = ['rare_nss', 'rare_auc', 'rare_ig', 'rare_cc', 'rare_kl', 'rare_sauc', 'rare_sim'])

In [35]:
#compare each participants fixations to qss saliency map and store output in dataframe

qss_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/qss_sal.png',0)
qss = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(qss_sal,df_fix.iloc[i][2])
    b = AUC(qss_sal,df_fix.iloc[i][2])
    c = IG(qss_sal,df_fix.iloc[i][2],dst)
    d = CC(qss_sal,blur)
    e = KLdiv(qss_sal,blur)
    f = SAUC(qss_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(qss_sal,blur)
    qss.append([a,b,c,d,e,f,g])

df_qss = pd.DataFrame(qss, columns = ['qss_nss', 'qss_auc', 'qss_ig', 'qss_cc', 'qss_kl', 'qss_sauc', 'qss_sim'])

In [36]:
#compare each participants fixations to lds saliency map and store output in dataframe

lds_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/lds_sal.png',0)
lds = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(lds_sal,df_fix.iloc[i][2])
    b = AUC(lds_sal,df_fix.iloc[i][2])
    c = IG(lds_sal,df_fix.iloc[i][2],dst)
    d = CC(lds_sal,blur)
    e = KLdiv(lds_sal,blur)
    f = SAUC(lds_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(lds_sal,blur)
    lds.append([a,b,c,d,e,f,g])

df_lds = pd.DataFrame(lds, columns = ['lds_nss', 'lds_auc', 'lds_ig', 'lds_cc', 'lds_kl', 'lds_sauc', 'lds_sim'])

In [37]:
#compare each participants fixations to imsig saliency map and store output in dataframe

imsig_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/imsig_sal.png',0)
imsig = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(imsig_sal,df_fix.iloc[i][2])
    b = AUC(imsig_sal,df_fix.iloc[i][2])
    c = IG(imsig_sal,df_fix.iloc[i][2],dst)
    d = CC(imsig_sal,blur)
    e = KLdiv(imsig_sal,blur)
    f = SAUC(imsig_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(imsig_sal,blur)
    imsig.append([a,b,c,d,e,f,g])

df_imsig = pd.DataFrame(imsig, columns = ['imsig_nss', 'imsig_auc', 'imsig_ig', 'imsig_cc', 'imsig_kl', 'imsig_sauc', 'imsig_sim'])

In [38]:
#compare each participants fixations to ikn saliency map and store output in dataframe

ikn_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/imsig_sal.png',0)
ikn = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(ikn_sal,df_fix.iloc[i][2])
    b = AUC(ikn_sal,df_fix.iloc[i][2])
    c = IG(ikn_sal,df_fix.iloc[i][2],dst)
    d = CC(ikn_sal,blur)
    e = KLdiv(ikn_sal,blur)
    f = SAUC(ikn_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(ikn_sal,blur)
    ikn.append([a,b,c,d,e,f,g])

df_ikn = pd.DataFrame(ikn, columns = ['imsig_nss', 'imsig_auc', 'imsig_ig', 'imsig_cc', 'imsig_kl', 'imsig_sauc', 'imsig_sim'])

In [39]:
#compare each participants fixations to gbvs saliency map and store output in dataframe

gbvs_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/gbvs_sal.png',0)
gbvs = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(gbvs_sal,df_fix.iloc[i][2])
    b = AUC(gbvs_sal,df_fix.iloc[i][2])
    c = IG(gbvs_sal,df_fix.iloc[i][2],dst)
    d = CC(gbvs_sal,blur)
    e = KLdiv(gbvs_sal,blur)
    f = SAUC(gbvs_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(gbvs_sal,blur)
    gbvs.append([a,b,c,d,e,f,g])

df_gbvs = pd.DataFrame(gbvs, columns = ['gbvs_nss', 'gbvs_auc', 'gbvs_ig', 'gbvs_cc', 'gbvs_kl', 'gbvs_sauc', 'gbvs_sim'])

In [40]:
#compare each participants fixations to gauss saliency map and store output in dataframe

gauss_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/gauss_sal.png',0)
gauss = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(gauss_sal,df_fix.iloc[i][2])
    b = AUC(gauss_sal,df_fix.iloc[i][2])
    c = IG(gauss_sal,df_fix.iloc[i][2],dst)
    d = CC(gauss_sal,blur)
    e = KLdiv(gauss_sal,blur)
    f = SAUC(gauss_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(gauss_sal,blur)
    gauss.append([a,b,c,d,e,f,g])

df_gauss = pd.DataFrame(gauss, columns = ['gauss_nss', 'gauss_auc', 'gauss_ig', 'gauss_cc', 'gauss_kl', 'gauss_sauc', 'gauss_sim'])

In [41]:
#compare each participants fixations to fes saliency map and store output in dataframe

fes_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/fes_sal.png',0)
fes = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(fes_sal,df_fix.iloc[i][2])
    b = AUC(fes_sal,df_fix.iloc[i][2])
    c = IG(fes_sal,df_fix.iloc[i][2],dst)
    d = CC(fes_sal,blur)
    e = KLdiv(fes_sal,blur)
    f = SAUC(fes_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(fes_sal,blur)
    fes.append([a,b,c,d,e,f,g])

df_fes = pd.DataFrame(fes, columns = ['fes_nss', 'fes_auc', 'fes_ig', 'fes_cc', 'fes_kl', 'fes_sauc', 'fes_sim'])

In [42]:
#compare each participants fixations to dva saliency map and store output in dataframe

dva_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/dva_sal.png',0)
dva = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(dva_sal,df_fix.iloc[i][2])
    b = AUC(dva_sal,df_fix.iloc[i][2])
    c = IG(dva_sal,df_fix.iloc[i][2],dst)
    d = CC(dva_sal,blur)
    e = KLdiv(dva_sal,blur)
    f = SAUC(dva_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(dva_sal,blur)
    dva.append([a,b,c,d,e,f,g])

df_dva = pd.DataFrame(dva, columns = ['dva_nss', 'dva_auc', 'dva_ig', 'dva_cc', 'dva_kl', 'dva_sauc', 'dva_sim'])

In [43]:
#compare each participants fixations to cvs saliency map and store output in dataframe

cvs_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/cvs_sal.png',0)
cvs = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(cvs_sal,df_fix.iloc[i][2])
    b = AUC(cvs_sal,df_fix.iloc[i][2])
    c = IG(cvs_sal,df_fix.iloc[i][2],dst)
    d = CC(cvs_sal,blur)
    e = KLdiv(cvs_sal,blur)
    f = SAUC(cvs_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(cvs_sal,blur)
    cvs.append([a,b,c,d,e,f,g])

df_cvs = pd.DataFrame(cvs, columns = ['cvs_nss', 'cvs_auc', 'cvs_ig', 'cvs_cc', 'cvs_kl', 'cvs_sauc', 'cvs_sim'])

In [44]:
#compare each participants fixations to cas saliency map and store output in dataframe

cas_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/cas_sal.png',0)
cas = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(cas_sal,df_fix.iloc[i][2])
    b = AUC(cas_sal,df_fix.iloc[i][2])
    c = IG(cas_sal,df_fix.iloc[i][2],dst)
    d = CC(cas_sal,blur)
    e = KLdiv(cas_sal,blur)
    f = SAUC(cas_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(cas_sal,blur)
    cas.append([a,b,c,d,e,f,g])

df_cas = pd.DataFrame(cas, columns = ['cas_nss', 'cas_auc', 'cas_ig', 'cas_cc', 'cas_kl', 'cas_sauc', 'cas_sim'])

In [45]:
#compare each participants fixations to dgi saliency map and store output in dataframe

dgi_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/dgi.png',0)
dgi_sal = cv2.resize(dgi_sal,(1920,1079))
dgi = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(dgi_sal,df_fix.iloc[i][2])
    b = AUC(dgi_sal,df_fix.iloc[i][2])
    c = IG(dgi_sal,df_fix.iloc[i][2],dst)
    d = CC(dgi_sal,blur)
    e = KLdiv(dgi_sal,blur)
    f = SAUC(dgi_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(dgi_sal,blur)
    dgi.append([a,b,c,d,e,f,g])

df_dgi = pd.DataFrame(dgi, columns = ['dgi_nss', 'dgi_auc', 'dgi_ig', 'dgi_cc', 'dgi_kl', 'dgi_sauc', 'dgi_sim'])

In [46]:
#compare each participants fixations to dgiie saliency map and store output in dataframe

dgiie_sal = cv2.imread('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/dgiie.png',0)
dgiie_sal = cv2.resize(dgiie_sal,(1920,1079))
dgiie = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(dgiie_sal,df_fix.iloc[i][2])
    b = AUC(dgiie_sal,df_fix.iloc[i][2])
    c = IG(dgiie_sal,df_fix.iloc[i][2],dst)
    d = CC(dgiie_sal,blur)
    e = KLdiv(dgiie_sal,blur)
    f = SAUC(dgiie_sal,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(dgiie_sal,blur)
    dgiie.append([a,b,c,d,e,f,g])

df_dgiie = pd.DataFrame(dgiie, columns = ['dgiie_nss', 'dgiie_auc', 'dgiie_ig', 'dgiie_cc', 'dgiie_kl', 'dgiie_sauc', 'dgiie_sim'])

In [47]:
#compare each participants fixations to dgii saliency map and store output in dataframe

dg2_cb_dens = np.load('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/dg2_cb_dens.npy')
dgii = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(dg2_cb_dens,df_fix.iloc[i][2])
    b = AUC(dg2_cb_dens,df_fix.iloc[i][2])
    c = IG(dg2_cb_dens,df_fix.iloc[i][2],dst)
    d = CC(dg2_cb_dens,blur)
    e = KLdiv(dg2_cb_dens,blur)
    f = SAUC(dg2_cb_dens,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(dg2_cb_dens,blur)
    dgii.append([a,b,c,d,e,f,g])

df_dgii = pd.DataFrame(dgiie, columns = ['dgii_nss', 'dgii_auc', 'dgii_ig', 'dgii_cc', 'dgii_kl', 'dgii_sauc', 'dgii_sim'])

In [48]:
#compare each participants fixations to icf saliency map and store output in dataframe

icf_cb_dens = np.load('C:/Users/njeri/Documents/Masters/Thesis/Scripts/Data/Maps/icf_cb_dens.npy')
icf = []

for i in range(len(df_fix)):
    blur = cv2.filter2D(df_fix.iloc[i][2],-1,kernel)
    a = NSS(icf_cb_dens,df_fix.iloc[i][2])
    b = AUC(icf_cb_dens,df_fix.iloc[i][2])
    c = IG(icf_cb_dens,df_fix.iloc[i][2],dst)
    d = CC(icf_cb_dens,blur)
    e = KLdiv(icf_cb_dens,blur)
    f = SAUC(icf_cb_dens,df_fix.iloc[i][2], shuf_map=f_map, step_size=.01)
    g = SIM(icf_cb_dens,blur)
    icf.append([a,b,c,d,e,f,g])

df_icf = pd.DataFrame(icf, columns = ['icf_nss', 'icf_auc', 'icf_ig', 'icf_cc', 'icf_kl', 'icf_sauc', 'icf_sim'])

In [49]:
df_fix.head()

,id,fixations,fixmap,aim_nss,aim_auc,aim_ig,aim_cc,aim_kl,aim_sauc,aim_sim
0,202107101214363,"[[288.8, 199.7], [1292.7, 915.9], [354.9, 946....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-0.173250,0.411120,-2.305416,-0.003296,5.056149,0.422562,0.000405
1,202107101432200,"[[891.4, 672.8], [1223.4, 744.5], [956.8, 167....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.224671,0.538412,-2.087863,0.004922,5.159385,0.560382,0.000518
2,202107101444503,"[[848.5, 851.7], [1400.1, 871.9], [1472.3, 295...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.686180,0.686619,-2.062538,0.014210,5.054440,0.714630,0.000539
3,202107101528378,"[[908.6, 304.9], [238.5, 226.8], [1354.1, 827....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.343402,0.564721,-2.492345,0.006796,5.055164,0.584228,0.000482
4,202107111045352,"[[921.4, 688.4], [117.9, 985.6], [211.4, 463.1...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-0.083830,0.428131,-2.281806,-0.002085,5.340200,0.434062,0.000555


In [52]:
#compile output to one dataframe
metrics = pd.concat([df_fix, df_rare, df_qss, df_lds, df_imsig, df_ikn, df_gbvs,df_gauss,df_fes, df_dva, df_cvs,df_cas, df_dgi,df_dgiie,df_dgii,df_icf],axis = 1)

In [54]:
#save dataframe to csv
metrics.to_csv('Data/metrics.csv')